In [4]:
import json
import pandas as pd
import xgboost as xgb
import joblib

# Load tracked token data files
def load_json(file_name):
    with open(file_name, 'r', encoding='utf-8') as file:
        return json.load(file)

files = [
    'data/coiny_bez_limitu12.json',
]

data = {}
for file in files:
    data.update(load_json(file))

# Convert JSON data to a DataFrame
def process_token_data(data):
    records = []
    for mint, token in data.items():
        for entry in token.get("marketCapHistory", []):
            timestamp = entry.get("timestamp")
            if timestamp > 10**10:  # Fix millisecond timestamps
                timestamp = timestamp / 1000
            records.append({
                "timestamp": timestamp,
                "marketCap": entry.get("marketCap", 0),
                "symbol": token.get("symbol", "Unknown"),
                "volume": token.get("volume", 0),
                "numHolders": token.get("numHolders", 0),
                "sniperCount": token.get("sniperCount", 0),
                "progress": token.get("progress", 0),
                "buySellRatio": token.get("buySellRatio", 0),
                "liquidity": token.get("liquidity", 0),
                "volatility": token.get("volatility", 0)
            })

        for neo in token.get("Neo", []):
            records.append({
                "symbol": token.get("symbol", "Unknown"),
                "holderCounts": neo.get("holderCounts", 0),
                "tagsCount": neo.get("tagsCount", 0),
                "devHoldingSupplyPerc": neo.get("devHoldingSupplyPerc", 0),
                "insiderWalletsSupplyPerc": neo.get("insiderWalletsSupplyPerc", 0)
            })

        for trades in token.get("trades", []):
            records.append({
                "timestamp": trades.get("timestamp"),
                "symbol": token.get("symbol", "Unknown"),
                "solAmount": trades.get("solAmount", 0),
                "isBuy": trades.get("isBuy", 0),
            })

    return pd.DataFrame(records)


df = process_token_data(data)

# Load trained models
buy_model = xgb.XGBClassifier(tree_method="hist", device="cuda")
buy_model.load_model('models/crypto_investment_model.json')

sell_model = xgb.XGBClassifier(tree_method="hist", device="cuda")
sell_model.load_model('models/crypto_sell_model.json')

# Load expected features
expected_features = ['marketCap_change', 'marketCap_ema', 'rsi', 'marketCap_change_ratio', 'holders_growth', 'volatility_to_volume_ratio']

# Function to align features
def align_features(df, expected_features):
    df_copy = df.copy()
    for feature in expected_features:
        if feature not in df_copy.columns:
            df_copy[feature] = 0
    return df_copy[expected_features]

# Track results
predictions_results = []

# Process each token
for token_key, token in data.items():

    if "marketCapHistory" not in token or not isinstance(token["marketCapHistory"], list):
        continue

    if len(token["marketCapHistory"]) < 10:
        continue

    market_caps = pd.DataFrame(token["marketCapHistory"])


    if market_caps.empty or "marketCap" not in market_caps.columns:
        continue

    market_caps['timestamp'] = pd.to_datetime(market_caps['timestamp'], errors="coerce")
    market_caps.sort_values('timestamp', inplace=True)
    market_caps.fillna(0, inplace=True)

    # Compute required features
    market_caps['marketCap_change'] = market_caps['marketCap'].pct_change().fillna(0)
    market_caps['marketCap_ema'] = market_caps['marketCap'].ewm(span=7).mean()
    market_caps['rsi'] = 100 - (100 / (1 + market_caps['marketCap'].pct_change().fillna(0).rolling(window=7).mean() /
                                      market_caps['marketCap'].pct_change().fillna(0).rolling(window=7).std())).fillna(0)
    market_caps['marketCap_change_ratio'] = ((market_caps['marketCap'].iloc[-1] - market_caps['marketCap'].iloc[0]) /
                                              market_caps['marketCap'].iloc[0]) * 100

    # Prepare features for buy model
    features_for_models = align_features(market_caps, expected_features)

    # Buy prediction, 95 is the confidence for buy action
    buy_prediction_probs = buy_model.predict_proba(features_for_models)[:, 1]
    current_price = market_caps['marketCap'].iloc[0]
    buy_threshold = 0.95

    if buy_prediction_probs[-1] > buy_threshold:
        buy_price = current_price
        highest_price = buy_price
        last_timestamp = market_caps['timestamp'].iloc[0]
        sold = False

        while True:
            updated_market_caps = pd.DataFrame(token["marketCapHistory"])
            updated_market_caps['timestamp'] = pd.to_datetime(updated_market_caps['timestamp'])
            updated_market_caps.sort_values('timestamp', inplace=True)
            new_data = updated_market_caps[updated_market_caps['timestamp'] > last_timestamp]

            if new_data.empty:
                break

            current_market_cap = new_data['marketCap'].iloc[0]
            highest_price = max(highest_price, current_market_cap)
            last_timestamp = new_data['timestamp'].iloc[0]

            drop_percentage = (highest_price - current_market_cap) / highest_price * 100
            updated_features = align_features(new_data, expected_features)
            sell_prediction_probs = sell_model.predict_proba(updated_features)[:, 1]

            # Confidence lower than 0.88 or the price of the token dropped by 5%
            if sell_prediction_probs[-1] > 0.88 or drop_percentage >= 5:
                predictions_results.append({
                    "symbol": token["symbol"],
                    "buy_price": buy_price,
                    "sell_price": current_market_cap,
                    "profit_percent": (current_market_cap - buy_price) / buy_price * 100
                })
                sold = True
                break

        if not sold:
            predictions_results.append({
                "symbol": token["symbol"],
                "buy_price": buy_price,
                "sell_price": None,
                "profit_percent": None
            })
    else:
        predictions_results.append({
            "symbol": token["symbol"],
            "buy_price": None,
            "sell_price": None,
            "profit_percent": None
        })

# Save results
results_df = pd.DataFrame(predictions_results)
results_df.to_csv('token_sell_predictions_updated.csv', index=False)
print("\nPredictions saved to token_sell_predictions_updated.csv")




Predictions saved to token_sell_predictions_updated.csv
